In [90]:
from mpi4py import MPI
from petsc4py.PETSc import ScalarType  # type: ignore

import numpy as np

import ufl
from dolfinx import fem, io, mesh, plot
from dolfinx.fem.petsc import LinearProblem
import dolfinx
import meshio


In [91]:
filename="meshes/square/mesh.vtu"
msh = meshio.read(filename)

meshio.write("meshes/square/mesh.mesh", msh)

In [92]:
# import meshio

# mesh_name = "meshes/square/mesh.msh"

# msh = meshio.read(mesh_name)

# tet_data = msh.cell_data_dict["gmsh:physical"]["tetra"]
# meshio.write(mesh_name+".xdmf",
#     meshio.Mesh(points=msh.points,
#         cells={"tetra": msh.cells_dict["tetra"]},
#         cell_data={"dom_marker": [tet_data]}
#     )
# )

# tri_data = msh.cell_data_dict["gmsh:physical"]["triangle"]
# meshio.write(mesh_name+"_surf.xdmf",
#     meshio.Mesh(points=msh.points,
#         cells={"triangle": msh.cells_dict["triangle"]},
#         cell_data={"bnd_marker": [tri_data]}
#     )
# )

In [93]:
filename="meshes/square/mesh.vtu"
msh = meshio.read(filename)

meshio.write("meshes/square/mesh.xdmf", msh)

In [94]:
filename="meshes/square/mesh.xdmf"
with dolfinx.io.XDMFFile(MPI.COMM_WORLD, filename, "r") as xdmf:
       msh = xdmf.read_mesh(name="Grid")

# msh = mesh.create_rectangle(
#     comm=MPI.COMM_WORLD,
#     points=((0.0, 0.0), (1.0, 1.0)),
#     n=(32, 16),
#     cell_type=mesh.CellType.triangle,
# )

print("Mesh topological dimension:", msh.topology.dim)
print("Mesh spatial dimension:", msh.geometry.dim)
print("Number of cells:", msh.topology.index_map(msh.topology.dim).size_global)
print("Number of vertices:", msh.topology.index_map(0).size_global)

Mesh topological dimension: 2
Mesh spatial dimension: 3
Number of cells: 944
Number of vertices: 2832


In [95]:
V = fem.functionspace(msh, ("Lagrange", 1))

In [96]:
facets = mesh.locate_entities_boundary(
    msh,
    dim=(msh.topology.dim - 1),
    marker=lambda x: np.isclose(x[0], 0.0) | np.isclose(x[0], 1.0),
)

dofs = fem.locate_dofs_topological(V=V, entity_dim=1, entities=facets)

bc = fem.dirichletbc(value=ScalarType(0), dofs=dofs, V=V)

print(facets)

[  1 116 119 125 131 137 143 149 155 161 167 173 179 185 191 197 203 209
 215 221 226 341 344 350 356 362 368 374 380 386 392 398 404 410 416 422
 428 434 440 446]


In [97]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)
x = ufl.SpatialCoordinate(msh)
# f = 10 * ufl.exp(-((x[0] - 0.5) ** 2 + (x[1] - 0.5) ** 2) / 0.02)
g = 10 * ufl.sin(6 * x[0])
a = ufl.inner(ufl.grad(u), ufl.grad(v)) * ufl.dx
# L = ufl.inner(f, v) * ufl.dx + ufl.inner(g, v) * ufl.ds

L = ufl.inner(g, v) * ufl.ds

In [ ]:
# There is a more manual method to solve this, similar to NGSolve, Heat_Equation_fenics.ipynb.

problem = LinearProblem(
    a,
    L,
    bcs=[bc],
    petsc_options_prefix="demo_poisson_",
    petsc_options={"ksp_type": "preonly", "pc_type": "lu", "ksp_error_if_not_converged": True},
)
uh = problem.solve()
assert isinstance(uh, fem.Function)

Error: error code 71
[0] KSPSolve() at /home/conda/feedstock_root/build_artifacts/bld/rattler-build_petsc_1761813293/work/src/ksp/ksp/interface/itfunc.c:1094
[0] KSPSolve_Private() at /home/conda/feedstock_root/build_artifacts/bld/rattler-build_petsc_1761813293/work/src/ksp/ksp/interface/itfunc.c:841
[0] KSPSetUp() at /home/conda/feedstock_root/build_artifacts/bld/rattler-build_petsc_1761813293/work/src/ksp/ksp/interface/itfunc.c:429
[0] PCSetUp() at /home/conda/feedstock_root/build_artifacts/bld/rattler-build_petsc_1761813293/work/src/ksp/pc/interface/precon.c:1120
[0] PCSetUp_LU() at /home/conda/feedstock_root/build_artifacts/bld/rattler-build_petsc_1761813293/work/src/ksp/pc/impls/factor/lu/lu.c:120
[0] MatLUFactorNumeric() at /home/conda/feedstock_root/build_artifacts/bld/rattler-build_petsc_1761813293/work/src/mat/interface/matrix.c:3336
[0] MatLUFactorNumeric_SeqAIJ_Inode() at /home/conda/feedstock_root/build_artifacts/bld/rattler-build_petsc_1761813293/work/src/mat/impls/aij/seq

In [ ]:
with io.XDMFFile(msh.comm, "output/square/poisson_sqaure_fenics.xdmf", "w") as file:
    file.write_mesh(msh)
    file.write_function(uh)